In [1]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0430 17:51:44.473403 140688525256448 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [4]:
!mkdir bert_output

mkdir: cannot create directory ‘bert_output’: File exists


In [5]:
OUTPUT_DIR = "bert_output"

In [6]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_output'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_output *****


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [7]:
from tensorflow import keras
import os
import re

In [8]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Capitalisation" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [9]:
modelname = "Capitalisation_ToFifteen" # @param {type:"string"}

In [10]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [11]:
len(trn)

12787

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [12]:
train = trn
test = tst

In [13]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [14]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [15]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [16]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
5        <bert.run_classifier.InputExample object at 0x...
6        <bert.run_classifier.InputExample object at 0x...
7        <bert.run_classifier.InputExample object at 0x...
8        <bert.run_classifier.InputExample object at 0x...
9        <bert.run_classifier.InputExample object at 0x...
10       <bert.run_classifier.InputExample object at 0x...
11       <bert.run_classifier.InputExample object at 0x...
12       <bert.run_classifier.InputExample object at 0x...
13       <bert.run_classifier.InputExample object at 0x...
14       <bert.run_classifier.InputExample object at 0x...
15       <bert.run_classifier.InputExample object at 0x...
16       <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [17]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 17:52:14.334359 140688525256448 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [18]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [19]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 12787


I0430 17:52:36.063642 140688525256448 tf_logging.py:115] Writing example 0 of 12787


INFO:tensorflow:*** Example ***


I0430 17:52:36.067844 140688525256448 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0430 17:52:49.593060 140688525256448 tf_logging.py:115] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0430 17:52:49.594350 140688525256448 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0430 17:52:49.595616 140688525256448 tf_logging.py:115] label: 0 (id = 0)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [20]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [21]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [22]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [23]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

2396


In [24]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [25]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_experimental_distribute': None, '_keep_checkpoint_max': 5, '_model_dir': 'bert_output', '_is_chief': True, '_protocol': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff3fae8c748>, '_evaluation_master': '', '_task_id': 0, '_num_ps_replicas': 0, '_device_fn': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': 500, '_tf_random_seed': None, '_save_checkpoints_secs': None, '_global_id_in_cluster': 0, '_train_distribute': None, '_service': None, '_log_step_count_steps': 100, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_master': ''}


I0430 17:52:52.678058 140688525256448 tf_logging.py:115] Using config: {'_save_summary_steps': 100, '_experimental_distribute': None, '_keep_checkpoint_max': 5, '_model_dir': 'bert_output', '_is_chief': True, '_protocol': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff3fae8c748>, '_evaluation_master': '', '_task_id': 0, '_num_ps_replicas': 0, '_device_fn': None, '_num_worker_replicas': 1, '_save_checkpoints_steps': 500, '_tf_random_seed': None, '_save_checkpoints_secs': None, '_global_id_in_cluster': 0, '_train_distribute': None, '_service': None, '_log_step_count_steps': 100, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_master': ''}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [26]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [27]:
print('Beginning Training!')
current_time = datetime.now()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/gpu:0'):
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0430 17:52:58.380560 140688525256448 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 17:53:01.585502 140688525256448 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 2396 into bert_output/model.ckpt.


I0430 18:09:29.430386 140688525256448 tf_logging.py:115] Saving checkpoints for 2396 into bert_output/model.ckpt.


INFO:tensorflow:Loss for final step: 0.0002477727.


I0430 18:09:33.003294 140688525256448 tf_logging.py:115] Loss for final step: 0.0002477727.


Training took time  0:16:40.304288


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [28]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [29]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0430 18:24:22.014785 140688525256448 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0430 18:24:26.017919 140688525256448 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 2396: auc = 0.829497, eval_accuracy = 0.9734126, f1_score = 0.75783473, false_negatives = 67.0, false_positives = 18.0, global_step = 2396, loss = 0.14906842, precision = 0.8807947, recall = 0.665, true_negatives = 2979.0, true_positives = 133.0


I0430 18:24:52.598196 140688525256448 tf_logging.py:115] Saving dict for global step 2396: auc = 0.829497, eval_accuracy = 0.9734126, f1_score = 0.75783473, false_negatives = 67.0, false_positives = 18.0, global_step = 2396, loss = 0.14906842, precision = 0.8807947, recall = 0.665, true_negatives = 2979.0, true_positives = 133.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2396: bert_output/model.ckpt-2396


I0430 18:24:55.351768 140688525256448 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 2396: bert_output/model.ckpt-2396


{'auc': 0.829497,
 'eval_accuracy': 0.9734126,
 'f1_score': 0.75783473,
 'false_negatives': 67.0,
 'false_positives': 18.0,
 'global_step': 2396,
 'loss': 0.14906842,
 'precision': 0.8807947,
 'recall': 0.665,
 'true_negatives': 2979.0,
 'true_positives': 133.0}

#Extracting the trained model


Now let's save our model:

In [0]:
# Export the model
def serving_input_fn():
  with tf.variable_scope("foo"):
    feature_spec = {
        "input_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "input_mask": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "segment_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
        "label_ids": tf.FixedLenFeature([], tf.int64),
      }
    serialized_tf_example = tf.placeholder(dtype=tf.string,
                                           shape=[None],
                                           name='input_example_tensor')
    receiver_tensors = {'examples': serialized_tf_example}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

EXPORT_DIR = './Exported models/'+modelname
estimator._export_to_tpu = False  # this is important
path = estimator.export_savedmodel(EXPORT_DIR, serving_input_fn)
print(path)

Check if we can load it correctly:

In [0]:
subdirs = [x for x in Path(EXPORT_DIR).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

In [0]:
from tensorflow.contrib import predictor

predict_fn = predictor.from_saved_model(latest)

In [30]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [31]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="blue">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [33]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/exam/exam2017_6/OBy_17_1.txt": "For illustrate come gives 2 pie charts, where we can see information about the transportation of goods between countries in Eastern Europe is 2008 year.\n\nFirstly, pie charts as soon, nomber onem what metals in goods transported by rail bigger number \u2013 35%, that 1/3 graphic. And can see, what in transported by road, food products about 35%, what is 1/3 of pie charts, but ne iaformation 30%. Selecting this 2 chartis, we can watching, what, chemicals and metols together 11%. In the pie charts is popular manufactured goods \u2013 28%. \n\nIn transported by road, machinery have a very smoll \u2013 2%. In 2 graphik, pie charts, merals and non-metallic minerals have a summarise information. \n\nI can see, summarise the information this two illustrote graphicL goods transported by rail and road, about of countries in Eastern Europe in 2008 year. In this charts transported we seen about pie charts, whe he illustrate, Eatch graphic have five pie. Now, transported is popular in our world.\n", "./data/exam/exam2017_6/OBy_123_2.txt": "Musicians and film producers lose huge amounts of money every year from illegal pirate copies. Copying and sharing music or films on the internet is theft and people who download these files, instead of buying the original discs, should be punished. To what extent do you agree or disagree?\n\nFirst of all, in modern age of internet and technologies people have fun in many ways and the most popular way is sharing music and films on the internet. \n\nIn the first place in this question should take into account the act, that many people can\u2019t afford to buy music or film. These people suppress their need with downloading and sharying films or music on the internet. These way, of course, is the most easy and comfortable. \n\nBut if you look from the producers and musicians it brings huge loses. Imagine that you did a great job and in the end you did not receive suffycient payment for these work. In this situation, anyone would feel cheated because any work and job, especially creative must be paid. After all, the creativity and arts bring people pleasure and very good to relax.\n\nIn the end, I am of the opinion that the films and music should be online, but it should not cost so much. Most often, the person comes to pay site and sees a large amount of licensed movie and thinks that the money he could to go and see that film at cinema. Therefore, I agree that online films and music can be pay, but they aren\u2019t be so expensive.\n", "./data/exam/exam2016/OR_55_1.txt": "The graf below represents the difference between the amount of children who did not get primary school education during the period of 2000-2012 by gender and region. The common trend is that there are more girls without primary school education then boys, except south Asia 2012 where this number of girls less then boys. We also can see that in each region there was a decline in the amount of children during the represented period. As it is evident from the bar chart there is a great difference between South Asia 2000 and South Asia 2012 equal to 6 millions in case of boys and about 16 millions in case of girls. The graf also shows that there are much more children without access to primary school education in Africa then in South Asia. The colums \u201crest of world\u201d give us an information that in Africa the number of described category is doubl the number in the rest of world, it is true as in 2000 as as in 2012. In Asia this amount became so small that it is now less then in the rest of world. To sum it all up, we can see from the bar chart that there is huge difference between the number of children without primary school education in different years and in different regions. The represented problem is a big problem of Africa especially.\n", "./data/exam/exam2014/EPa_38_2.txt": "Many people believe that increasing of the number of sports facilities improve public health. However, there are many other people who say that it can not help to improve public health. Who is right? \nLet`s consider this question for all sides. Suppose that the number of sports facilities was increased. But how will we know that people are interesting it? It is can do if make survey people and know thir interests. \nExample: ask children that is their like sport play. And after analysis answers will build sport area for this play. also we can ask other people and increase quantity of sport areas. I think tat it should improve public health.\nOn other hand many people do not want to engage sport, and like just sit at home. To solve this problem we should make motivation for those people. We should advate sport and sport activity or equpment. Example: boots and clothes for morning running. If people will see on display their TV such, ads they will think that healthy life and engage sports is better than sit at home.\nIn conclusion I want t say that men self create own life and if he want to be health, will engage sport.\n", "./data/exam/undefined/DZu_156_1.txt": "The bar chart gives information about amount of younger people without access to primary education in various parts of World (Africa, South Asia, Rest of World) with gender information during the period from 2000 to 2012. Units are measured in millions. Overall, the bar chart clearly shows that worst situation was is in Africa and South Asia in 2000. In Africa numbers was nearly 45 million of children whilst in South Africa just under 30 millions. As it may be seen from the graph, the total number in these regions show decreasing trend.\nThe total number of children without access to primary school was shaking going down in South Asia. The bar chart plainly indicates that amount of these people reduced more than three times to 10 millions in 2012. Another important trend is the gender data in South Asia that demonstrates a nunbers to boys (5,1 millions) and girls (4,8). The bar chart provides strongly evidence that a total nunber of children without access to primary education in the Earth gently goes down and these points shows a \u201cRest of World\u201d burs. In comparison, these indicators declines from approximately 25 millions in 2000 just under 15 millions in 2012.\n", "./data/exam/exam2017/EGe_129_2.txt": "During the last decades an overwhelming majority of offences are committed by people who are under the age. This pronlem is becoming more and more serious in our days so it is important to take some measures to solve it. These essay will provide some causes of this issue and outline ideas to tackle it.\nFirst of all, some young people do not have any hobbies and interests so to make their lifes a little bit more exciting they start to break rules. For example, if parents are busy at their work and sometimes work overtime, there is nobody to take care after their child. As a resuit, these kids do what they want like watching TV, playing computer games and so on. After that they can go for a walk so as to make his life wore exciting and to commite some crimes like stealing money or food from the shop especially if his patents do not give him any pocket money. Nevertheless, This problem can be simply tackled. Parents should organize some extra activities for their childen and to make sure they are busy enough. On the other hand, teachers working at schools can forge some clubs of interest like English learning or History extra classes.\nFurthermore, Some children do some bad things listening to their friends, classmates or somebody who is older than they are. for example,if a child wants to make some new friends who are older that he is, he can mug an old woman and steal her money in order to be respected by his new friends. In that case, parents should know every single friend of his child and to be sure that they do not have a negative impact on his kid while teachers should to pay more attention on relationships in theit class. In conclusion, even through the rates of crime comitted by children are increasing, it is possible to decrease them or make them less serious.", "./data/exam/exam2016/OR_19_1.txt": "There are two charts which show information about amount of facebook\u2019s users and also user\u2019s reasons for using facebook. On the first diagramm we can see that desktop\u2019s users have downward trend in period March 2012 to March 2013. Moreover. Less than 140 millions people use facebook on desktop. In spite of this fact amount of people who use mobile version grew sharply.\n", "./data/exam/exam2017_7/VSa_23_1.txt": "Thes pe charts give information about the transportation of goods between countries in Eastern Europe in 2008. Globally, Goods transported by rail has a 25% manufactured goods, 35% metals, 12% machinery and 11% chemicals and food products.\n", "./data/exam/exam2017_7/VSa_3_1.txt": "This pie charts illustrate the information of goods between countries in Eastern Europ in 2008. It's clear that the largest of Metals 35% transported by rail. but lot's of Fod products transorted by road.\nIn 2008 12% Machinery, 25 Manufactured goods 25%, Metals 35%. Food product\u2019s 11%. and 11% Chemicals transported by rail\n", "./data/old IELTS/IELTS2015/ASt_11_1.txt": "The graph given illustrates the percentage difference in 65 aged population between Japan, Sweden and the United States. All the data included is from 1940 to 2040 year.\nOverall, even from the first look, we can spot the brilliant progress of Japan which used to be the worst of these three countries in 2040 and managed to become the first by 2040. All in all, all the countries given showed a positive trend in general and they had significantly increased the rate of 65 aged people.Looking each of them more precisely, we can see some periods, when this rate was declined. For instance, in Japan the percentage rate of elderly people dropped from 5% to approximately 2,5% from 1940 to 1990, however, after this it becomes slowly rising and rocketed approximately from 2030 hitting a peak by 2040 at the point of 27%. The curves, belonged to the USA and Sweden went similarly until 1990, than there was a significant growth of elderly people rate in Sweden, which fluctuated from 2010 to 2020 and then continued to rise. In the USA, in turn there was a signification for 40 years (from 1980 to 2020) and them during the last 20 year period there was a sharp rise. \n", "./data/exam/exam2017/DPe_29_1.txt": "The chart deals with the statistics of unemployment in six parts of the world in 2014 and 2015.\nAccording to the graph, North Africa had the highest unemployment rate in 2014. The number of unemployment of South Asia was the lowest. It was 3,9% while Middle East showed 11% rate of unemployment which was higher than in The EU and Latin America in 2014. They showed 7,5% and 6,8 accordingly.\nThe chart clearly shows that many regions maintained at the same level of unemployment in 2015. For instance, North Africa, South Africa remained steadily. Middle Eeast and Latin America illustrated the sharp fall of enemployment in 2015. It means that some people have found the place of work. However, European Union demonstrated upward trend of unemployment. The number of unemployed people has grown from 7,5% to 7,9% in European countries.\nTo conclude, instead of gradual growth of employed people in Latin America and Middle East, the statistics of Worldwide number of unemployment did not change to the year of 2015 and present 5,9%. It means that unemployment is a global problem which should be solved.\n", "./data/exam/exam2017/ABl_44_1.txt": "The bar chart includes data about total percentage of unemployed people in regions of the world in 2014 and 2015. The total unemployment rate remains stable around the whole world by 5,9 percent in 2-years period. The biggest amounts are in South Asia (3,9%). There are two regions that we indicated by decrease of unemployment rate during the two-years period. In Middle East the drop was from 11% to 9,6% in comparison with the drop in Latin America from 6,8% to 5,8%. the EU\u2019s unemployment rate increase steadily by 0,4 percentage points. The level provide clear evidence, that the world regions have very different amounts of unemployment rate in 2014 and 2015. The amounts of unemployment rate remain stable in two regions of the world while two regions are characterized by drop of such amount and only one region as an increased unemployment rate in 2015 in comparison with 2014 (the EU).\n", "./data/exam/exam2014/VPe_27_1.txt": "The bar charts Illustrate the number of students from under 26 to over 49 years old in connection with their motives for entering universities and also the amount of maintenance they are given at work from their chefs.\nAs for the first diagram, it is clear that the quantity of people under 26 studying for the sake of career is 80%, which prevails that if those who study out of interest, which is only 40%. In the meantime, the result for students over 49 years old is quite opposite: 70% go to universities because they are interested, and only 20% are inspired to do so to built a career. As soon as other generational groups are concerned, the percentage of people studying for career purposes is higher in all cases, except for students bet went 40 and 50 years old, for which the percentage is divided equally.\nThe second bar chart , showing support the students are supplies with by their employers p, demonstrates a fluctuation starting at approximately 60% for people under 26 , it reaches its lowest of 30% for students at the age of 30 - 39 and then increases steadily to upward 40% for the oldest group.\nTo sum up, there is a tendency that, getting older, people go into higher education more frequently out of interest rather than for career and get less support from the chef at work, as compared to the youngest group.\n", "./data/exam/exam2017_4/DPe_59_2.txt": "Musical and film industry lose a lots of money in a modern times because of illegal copies and \u201cpirates\u201d attacks. The problem is that people nowadays prefer download some films and music albums from internent. As a result, musicians and film producers lose their money, because none of people want to buy their works.\nAs for me, I agree with this point of view. And I also will try to prove it. The first thing is, that musicians and film producers waste their time and nerves while makeing something new. A lot of my friends are musicians and they say that it is realy hard to record something good without having enough maney. Imagine how wasteful the film is: operators work, paying to actors etc. Pirate copies make musicians and film producers less motivated, because in the end they get almost no money for their work. The second thing is, that every musician need money. It is well known fact that domestic recordings are not so good. Recording in a studio costs a lot of money; the same thing with mastering a composition or song. Musicians hope that they will sell their recording for example in i-tunes and earn some money to make new music and new albums. This second fact shows that music industry is too wasteful like cinema and pirate copies destroy the enthusiasm of musician. The last fact about such topic is that concerts don\u2019t give enough money to cover all wastings. We can say that cinema industry has the same problems too.\nBut some people say that they don\u2019t need money to produce something new. For example in underground culture money don\u2019t play an important role. The concept of underground may seem strange. Films and music contains strong language and forbidden topics. It is clear that people won\u2019t pay any money for that. Pirate copies are not forbidden in underground; reverse, this culture adore this. Illegal copies make free advertisment for some musicians.\nTo sum up, I would like to say that illegal copies influence really bad on film producers and some musicians because as we can know from above these two industries needs a lot of money. But for others, pirate copies seems so good. It\u2019s their point of view. As for me, (as I said before) illegal copies will be always bad thing in internet.\n", "./data/old IELTS/IELTS2015/AMe_7_1.txt": "The graph illustrates the persantage of post school qualifications and the different between percantage of males and females according to 1999.\n\tWe can see that the largest percantage among males is skilled vocational diploma. It is more 90%. And according with this graph we can see that the large percantage among female is 72%. And this figures are undergraduate diploma. But if we wanted to know what the most low percantages of femails in post-school qualifications. we must to see the postgraduate diploma. We can see only 30% . I think that it can associated with that fact and educate their. And also we can see that the most low percantages of males is thouse man who have undergraduate diploma.\n\tAnd as we can see that the other positions are located in the midle of this graph and less interesting for our study.\n"}
outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 128


I0430 18:28:59.612332 140688525256448 tf_logging.py:115] Writing example 0 of 128


INFO:tensorflow:*** Example ***


I0430 18:28:59.615763 140688525256448 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0430 18:30:42.212257 140688525256448 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0430 18:30:42.295249 140688525256448 tf_logging.py:115] Done running local_init_op.




Annotation took time  0:00:06.984611


In [37]:
os.listdir(".")

['.ipython',
 '.cache',
 '.ipynb_checkpoints',
 'delete.json',
 '.jupyter',
 '.bashrc-anaconda3.bak',
 '.config',
 'REALEC_AutoAnnotator_BERT_pipeline Often_confused.ipynb',
 'wsites_OftenConfused_ToFifteen.json',
 'Often_confused_test.json',
 'Often_confused.json',
 'anaconda3',
 'Anaconda3-4.0.0-Linux-x86_64.sh',
 'download_google_drive',
 '.nano',
 '.bash_history',
 'Capitalisation.json',
 'gdrive-linux-x64',
 '.keras',
 'delete_test.json',
 '.python_history',
 'nltk_data',
 'wsites_delete_ToFifteen.json',
 'wsites_Capitalisation_ToFifteen.json',
 'Datasets',
 '.local',
 'wsites_lex_item_choice_AugmentedRatio.json',
 'REALEC_AutoAnnotator_BERT_pipeline Capitalisation.ipynb',
 'jupyter-drive',
 'cuda-repo-ubuntu1604_9.0.176-1_amd64.deb',
 '.continuum',
 '.conda',
 '.bashrc',
 '.nv',
 'bert_output',
 'Capitalisation_test.json',
 'cuda-repo-ubuntu1604_8.0.61-1_amd64.deb',
 '.gnupg',
 'REALEC_AutoAnnotator_Generic_BERT_pipeline Delete.ipynb',
 '.gdrive',
 'lex_item_choice_test.json',
 '

In [35]:
outie[4]

'<html>\n<head>\n<title>./data/exam/undefined/DZu_156_1.txt</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>The bar chart gives information about amount of younger people without access to primary education in various parts of <div class="blue">World</div> ( Africa, South Asia, <div class="blue">Rest</div> of <div class="blue">World</div>) with gender information during the period from 2000 to 2012. <div class="blue">Units</div> are measured in millions. Overall, the bar chart clearly shows that worst situation was is in Africa and South Asia in 2000. In Africa numbers was nearly 45 million of children whilst in South Africa just under 30 millions. As it may be seen from the graph, the total number in these regions show decreasing trend. The total number of children without access to primary school was shaking going down in South Asia. The bar chart plainly indicates that amount of these pe

In [36]:
!./gdrive-linux-x64 upload wsites_Capitalisation_ToFifteen.json

Uploading wsites_Capitalisation_ToFifteen.json
Uploaded 1QxRvnI7eft_dR9poIdE6wqsOfspjgvth at 14.0 KB/s, total 21.3 KB


In [38]:
!ls -lAh ./bert_output

total 6.5G
-rw-rw-r-- 1 ivantorubarov ivantorubarov  270 Apr 30 18:09 checkpoint
drwxr-xr-x 2 ivantorubarov ivantorubarov 4.0K Apr 30 18:24 eval
-rw-rw-r-- 1 ivantorubarov ivantorubarov  48M Apr 30 17:28 events.out.tfevents.1556645336.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov  48M Apr 30 17:43 events.out.tfevents.1556646090.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov  96M Apr 30 18:09 events.out.tfevents.1556646804.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov  86M Apr 30 17:53 graph.pbtxt
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 18:00 model.ckpt-1000.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K Apr 30 18:00 model.ckpt-1000.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  25M Apr 30 18:00 model.ckpt-1000.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G Apr 30 18:03 model.ckpt-1500.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K Apr 30 18:03 model.ckpt-1500.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  25M Apr

In [ ]:
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.data-00000-of-00001
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.index
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.meta

Uploading ./bert_output/model.ckpt-2396.data-00000-of-00001
234.9 MB/1.3 GB, Rate: 41.1 MB/s                  

In [1]:
!rm -rf ./bert_output

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd


# Preparing existing model for REALEC production